In [11]:
import os
import json
import base64
from base64 import b64decode
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.x509 import load_pem_x509_certificate
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes, serialization
import ssl
import socket
import nbimporter
import threading
import traceback
from AutoridadeCertificadora2 import AutoridadeCertificadora as CA

In [13]:
def gerar_chave_aes():
    return AESGCM.generate_key(bit_length=256)

def criptografar_mensagem(chave_aes, mensagem):
    aesgcm = AESGCM(chave_aes)
    nonce = os.urandom(12)
    cifrada = aesgcm.encrypt(nonce, mensagem.encode(), None)
    return nonce, cifrada

def criptografar_chave_aes(chave_aes, pub_destinatario):
    return pub_destinatario.encrypt(
        chave_aes,
        padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
    )

def desencriptar_mensagem(mensagem, path_chave_privada):
    
    with open(path_chave_privada, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
                       key_file.read(),
                       password=None, 
              )
    dados = json.loads(mensagem)
    nonce = b64decode(dados['nonce'])
    chave_cifrada = b64decode(dados['chave_cifrada'])
    mensagem_cifrada = b64decode(dados['mensagem_cifrada'])

    chave_simetrica = private_key.decrypt(
        chave_cifrada,
        padding.OAEP(
           mgf=padding.MGF1(algorithm=hashes.SHA256()),
           algorithm=hashes.SHA256(),
           label=None
        )
    )

    aesgcm = AESGCM(chave_simetrica)
    mensagem_desencriptada = aesgcm.decrypt(nonce, mensagem_cifrada, None)
    return mensagem_desencriptada.decode('utf-8')

def enviar_pacote_json(chave_aes, mensagem, chave_pub_destinatario):
    nonce, cifrada = criptografar_mensagem(chave_aes, mensagem)
    chave_aes_cifrada = criptografar_chave_aes(chave_aes, chave_pub_destinatario)
    pacote = {
           "nonce": base64.b64encode(nonce).decode(),
           "mensagem_cifrada": base64.b64encode(cifrada).decode(),
           "chave_cifrada": base64.b64encode(chave_aes_cifrada).decode()
    }
    return json.dumps(pacote).encode()

def validacoes(cert, cert_CA, nome_esperado):
    return (CA().is_valido(cert) and CA().is_autentico(cert, cert_CA) and CA().is_identidade_valida(cert, nome_esperado) and not(CA().is_revogado(cert)))

def chaves_clientes_grupo(arr_clientes):
    chave_gsk = gerar_chave_aes()
    dicionario = {}

    with open("gsk_original.pem", "wb") as f:
       f.write(chave_gsk)
    
    for cliente in arr_clientes:
        path = f"certificados/{cliente}_cert.pem"
        if not(os.path.exists(path)):
            print(f"O cliente {cliente} não existe.")
            continue
        with open(path, "rb") as f:
           cert_dest = load_pem_x509_certificate(f.read())
           chave_pub_dest = cert_dest.public_key()
        chave_gsk_encriptada = criptografar_chave_aes(chave_gsk, chave_pub_dest)
        dicionario[cliente] = chave_gsk_encriptada
        
    return dicionario

def desencriptar_gsk(gsk_cifrada, path_chave_privada):
    with open(path_chave_privada, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None
        )
    
    chave_gsk = private_key.decrypt(
        gsk_cifrada,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return chave_gsk

def desencriptar_mensagem_grupo(mensagem_bytes, gsk_bytes):
    aesgcm = AESGCM(gsk_bytes)
    # O cliente precisa enviar nonce + ciphertext + tag
    # Supondo que a mensagem foi cifrada com AESGCM.encrypt(nonce, plaintext, None)
    # então a tag já está embutida no ciphertext no final (padrão do cryptography)
    nonce = mensagem_bytes[:12]  # primeiros 12 bytes
    ciphertext = mensagem_bytes[12:]
    return aesgcm.decrypt(nonce, ciphertext, None).decode('utf-8')

def enviar_mensagem_grupo(mensagem, gsk):
    aesgcm = AESGCM(gsk)
    nonce = os.urandom(12)
    ciphertext = aesgcm.encrypt(nonce, mensagem.encode('utf-8'), None)
    return nonce + ciphertext

def carregar_gsk(path_gsk):
    with open(path_gsk, "rb") as f:
        return f.read()

def ouvir_mensagens(conn, gsk_desencriptada):
    while True:
        try:
            resposta = conn.recv(4096)
            if not resposta:
                print("❌ Conexão encerrada pelo servidor.")
                break
            mensagem = desencriptar_mensagem_grupo(resposta, gsk_desencriptada)
            print(mensagem, end="", flush=True)
        except Exception as e:
            print("⚠️ Erro ao receber mensagem:", e)
            break

def enviar_mensagens(conn, gsk_desencriptada, nome_cliente):
    try:
        while True:
            texto = input("Você: ")
            if texto.lower() == "sair":
                break
            mensagem = f"{nome_cliente}: {texto}"
            mensagem_cifrada = enviar_mensagem_grupo(mensagem, gsk_desencriptada)
            conn.sendall(mensagem_cifrada)
    except Exception as e:
        print("⚠️ Erro ao enviar mensagem:", e)

def main_cliente(nome_cliente):
    PATH_CERT = f"certificados/{nome_cliente}_cert.pem"
    PATH_CHAVE = f"certificados/{nome_cliente}_chave.pem"
    PATH_CA = "AutoridadeCertificadora/certificado_raiz.pem"
    PATH_GSK = f"gsk_encriptada_para_{nome_cliente}.pem"

    if not os.path.exists(PATH_GSK):
        print("❌ Você não foi autorizado a entrar no grupo (GSK não encontrada).")
        return

    if not validacoes(PATH_CERT, PATH_CA, nome_cliente):
        print("❌ Certificado digital inválido ou não autorizado.")
        return

    gsk_encriptada = carregar_gsk(PATH_GSK)
    gsk_desencriptada = desencriptar_gsk(gsk_encriptada, PATH_CHAVE)

    context = ssl.create_default_context()
    context.load_cert_chain(certfile=PATH_CERT, keyfile=PATH_CHAVE)
    context.load_verify_locations(cafile=PATH_CA)
    context.verify_mode = ssl.CERT_REQUIRED

    with socket.create_connection(('localhost', 4444)) as sock:
        with context.wrap_socket(sock, server_hostname="Marianna") as conn:
            print("✅ Conectado ao grupo com sucesso.")

            thread_ouvir = threading.Thread(target=ouvir_mensagens, args=(conn, gsk_desencriptada), daemon=True)
            thread_ouvir.start()

            enviar_mensagens(conn, gsk_desencriptada, nome_cliente)
